# 1. Import and Install Dependencies

In [1]:
#pip install tensorflow==2.7.1 tensorflow-gpu==2.7.1 opencv-python==4.5.1.48 mediapipe==0.10.5 scikit-learn==0.24.2 numpy==1.19.5 matplotlib         

In [1]:
# import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                             ) 

In [6]:
cap = cv2.VideoCapture(1) ### chage to your device usually 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        # print(frame.shape)
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
draw_landmarks(frame, results)

In [8]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))s

SyntaxError: invalid syntax (544629361.py, line 1)

# 3. Extract Keypoint Values

In [9]:
results.left_hand_landmarks

In [10]:
len(results.left_hand_landmarks.landmark)

AttributeError: 'NoneType' object has no attribute 'landmark'

In [11]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [12]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [14]:
result_test = extract_keypoints(results)

In [15]:
result_test

array([ 6.29487872e-01,  2.19172046e-01, -6.54516876e-01,  9.98675168e-01,
        6.34286702e-01,  1.69778928e-01, -6.03439689e-01,  9.97711360e-01,
        6.44172609e-01,  1.68240637e-01, -6.03711486e-01,  9.98169065e-01,
        6.52658403e-01,  1.66323423e-01, -6.04214966e-01,  9.97844040e-01,
        5.98716617e-01,  1.74062967e-01, -6.25518918e-01,  9.98145044e-01,
        5.82888663e-01,  1.76917464e-01, -6.25381589e-01,  9.98613834e-01,
        5.67175448e-01,  1.81031153e-01, -6.25975490e-01,  9.98274684e-01,
        6.56001747e-01,  1.86152175e-01, -2.77966619e-01,  9.98543262e-01,
        5.36487937e-01,  2.06216097e-01, -3.64572704e-01,  9.98942316e-01,
        6.51735604e-01,  2.73733139e-01, -5.30587375e-01,  9.98975515e-01,
        6.04211509e-01,  2.79718876e-01, -5.54569125e-01,  9.99118209e-01,
        7.66258657e-01,  4.36114758e-01, -1.57102831e-02,  9.99551594e-01,
        4.45582867e-01,  4.50312287e-01, -2.48172492e-01,  9.98707592e-01,
        9.26791310e-01,  

In [16]:
np.save('0', result_test)

In [17]:
np.load('0.npy')

array([ 6.29487872e-01,  2.19172046e-01, -6.54516876e-01,  9.98675168e-01,
        6.34286702e-01,  1.69778928e-01, -6.03439689e-01,  9.97711360e-01,
        6.44172609e-01,  1.68240637e-01, -6.03711486e-01,  9.98169065e-01,
        6.52658403e-01,  1.66323423e-01, -6.04214966e-01,  9.97844040e-01,
        5.98716617e-01,  1.74062967e-01, -6.25518918e-01,  9.98145044e-01,
        5.82888663e-01,  1.76917464e-01, -6.25381589e-01,  9.98613834e-01,
        5.67175448e-01,  1.81031153e-01, -6.25975490e-01,  9.98274684e-01,
        6.56001747e-01,  1.86152175e-01, -2.77966619e-01,  9.98543262e-01,
        5.36487937e-01,  2.06216097e-01, -3.64572704e-01,  9.98942316e-01,
        6.51735604e-01,  2.73733139e-01, -5.30587375e-01,  9.98975515e-01,
        6.04211509e-01,  2.79718876e-01, -5.54569125e-01,  9.99118209e-01,
        7.66258657e-01,  4.36114758e-01, -1.57102831e-02,  9.99551594e-01,
        4.45582867e-01,  4.50312287e-01, -2.48172492e-01,  9.98707592e-01,
        9.26791310e-01,  

# 4. Setup Folders for Collection

In [101]:
# Path for exported data, numpy arrays
from pathlib import Path
DATA_PATH = Path(r'C:\Users\MANIA\Desktop\grad_project\MP_Data') ### change your path

# Actions that we try to detect
actions = np.array([ 'fire', 'clean', 'dirty', 'give me', 'small', 'big', 'sho dakhalak'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 45

# Folder start
start_folder = 0

In [69]:
for action in actions: 
    
    for sequence in range(start_folder,no_sequences+1):
        try: 
            os.makedirs(Path.joinpath(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [70]:
cap = cv2.VideoCapture(1) ### chage to your device usually 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences + 1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = Path.joinpath(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.waitKey(1900)
    cap.release()
    cv2.destroyAllWindows()


In [71]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [102]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [103]:
label_map = {label:num for num, label in enumerate(actions)}

In [104]:
label_map

{'fire': 0,
 'clean': 1,
 'dirty': 2,
 'give me': 3,
 'small': 4,
 'big': 5,
 'sho dakhalak': 6}

In [105]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [106]:
np.array(sequences).shape

(427, 45, 258)

In [107]:
np.array(labels).shape

(427,)

In [108]:
X = np.array(sequences)

In [109]:
X.shape

(427, 45, 258)

In [110]:
y = to_categorical(labels).astype(int)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [112]:
y_test.shape

(22, 7)

# 7. Build and Train LSTM Neural Network

In [113]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [114]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [115]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(45,258)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [116]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [117]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
13/13 [==============================] - 4s 53ms/step - loss: 1.5892 - categorical_accuracy: 0.3235
Epoch 2/100
13/13 [==============================] - 1s 52ms/step - loss: 1.2770 - categorical_accuracy: 0.4000
Epoch 3/100
13/13 [==============================] - 1s 51ms/step - loss: 0.9939 - categorical_accuracy: 0.6099
Epoch 4/100
13/13 [==============================] - 1s 52ms/step - loss: 0.7248 - categorical_accuracy: 0.7432
Epoch 5/100
13/13 [==============================] - 1s 54ms/step - loss: 0.5891 - categorical_accuracy: 0.7852
Epoch 6/100
13/13 [==============================] - 1s 52ms/step - loss: 0.5658 - categorical_accuracy: 0.7630
Epoch 7/100
13/13 [==============================] - 1s 52ms/step - loss: 0.4680 - categorical_accuracy: 0.7901
Epoch 8/100
13/13 [==============================] - 1s 51ms/step - loss: 0.4694 - categorical_accuracy: 0.7753
Epoch 9/100
13/13 [==============================] - 1s 47ms/step - loss: 0.7058 - categorical_accuracy:

In [118]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 45, 64)            82688     
                                                                 
 lstm_9 (LSTM)               (None, 45, 128)           98816     
                                                                 
 lstm_10 (LSTM)              (None, 45, 128)           131584    
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)               

# 8. Make Predictions

In [119]:
res = model.predict(X_test)

In [120]:
actions[np.argmax(res[4])]

'clean'

In [121]:
actions[np.argmax(y_test[4])]

'clean'

# 9. Save Weights

In [122]:
model.save('action.h5')

In [123]:
model.load_weights('./action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [124]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [125]:
yhat = model.predict(X_test)

In [126]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [127]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[19,  0],
        [ 0,  3]],

       [[18,  0],
        [ 0,  4]],

       [[19,  0],
        [ 0,  3]],

       [[19,  0],
        [ 0,  3]],

       [[17,  0],
        [ 0,  5]],

       [[19,  0],
        [ 0,  3]],

       [[21,  0],
        [ 0,  1]]], dtype=int64)

In [128]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [129]:
from scipy import stats

In [130]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,137,205), (16,17,245), (122,147,45), (156,117,240)]*5
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [131]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-45:]
        
        if len(sequence) == 45:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [65]:
cap.release()
cv2.destroyAllWindows()

In [6]:
import numpy as np
import cv2
import mediapipe as mp
from model import create_model

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


actions = np.array(['null','besm allah' , 'alsalam alekom' , 'alekom salam' , 'aslan w shlan' , 'me',
                    'age','alhamdulilah' , 'bad' , 'how are you' , 'friend' ,
                    'good' , 'happy' , 'you' , 'my name is' , 'no' , 
                    'or' , 'taaban' , 'what' , 'where' , 'yes' ,
                    'look' , 'said' , 'walking' , 'did not hear' , 'remind me',
                    'eat' , 'bayt' , 'hospital' , 'run' , 'sleep',
                    'think' , 'tomorrow' , 'yesterday' , 'today' , 'when',
                    'dhuhr' , 'sabah' , 'university' , 'kuliyah' ,'night',
                    'a3ooth bellah' , 'danger' , 'enough' , 'hot' , 'mosque' , 'surprise' , 'tard' , 
                    'big' , 'clean' , 'dirty' , 'fire' , 'give me' , 'sho dakhalak' , 'small' , 
                    'help' , 'same' , 'hour' , 'important' , 'ok' , 'please' , 'want' ,
                    'riyadah' , 'sallah' , 'telephone' , 'hamam' , 'water' , 'eid'
                   ])

label_map = {label:num for num, label in enumerate(actions)}

def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                             ) 

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

colors = [(245,117,16), (117,245,16), (16,117,245),(16,137,205), (16,17,245), (122,147,45), (156,117,240)]*10
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5
model = create_model(actions.shape[0])
model.load_weights('action.h5')


cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-45:]
        
        if len(sequence) == 45:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()